# Indexación y selección de datos

En la [Parte 2](02.00-Introducción-a-NumPy.ipynb), analizamos en detalle los métodos y herramientas para acceder, establecer y modificar valores en matrices NumPy.
Estos incluían indexación (por ejemplo, `arr[2, 1]`), división (por ejemplo, `arr[:, 1:5]`), enmascaramiento (por ejemplo, `arr[arr > 0]`), indexación elegante (por ejemplo, , `arr[0, [1, 5]]`), y combinaciones de los mismos (por ejemplo, `arr[:, [1, 5]]`).
Aquí veremos medios similares para acceder y modificar valores en los objetos Pandas `Series` y `DataFrame`.
Si ha utilizado los patrones NumPy, los patrones correspondientes en Pandas le resultarán muy familiares, aunque hay algunas peculiaridades que debe tener en cuenta.

Comenzaremos con el caso simple del objeto `Series` unidimensional y luego pasaremos al objeto `DataFrame` bidimensional más complicado.

## Selección de datos en serie

Como vio en el capítulo anterior, un objeto `Series` actúa en muchos sentidos como una matriz NumPy unidimensional y, en muchos sentidos, como un diccionario estándar de Python.
Si tiene en cuenta estas dos analogías superpuestas, le ayudará a comprender los patrones de indexación y selección de datos en estas matrices.

### Serie como diccionario

Como un diccionario, el objeto `Series` proporciona una asignación de una colección de claves a una colección de valores:

In [1]:
import pandas as pd
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [2]:
data['b']

0.5

También podemos usar expresiones y métodos de Python similares a un diccionario para examinar las claves/índices y valores:

In [3]:
'a' in data

True

In [4]:
data.keys()

Index(['a', 'b', 'c', 'd'], dtype='object')

In [5]:
list(data.items())

[('a', 0.25), ('b', 0.5), ('c', 0.75), ('d', 1.0)]

Los objetos `Series` también se pueden modificar con una sintaxis similar a un diccionario.
Así como puedes extender un diccionario asignándolo a una nueva clave, puedes extender una `Serie` asignándole un nuevo valor de índice:

In [6]:
data['e'] = 1.25
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

Esta fácil mutabilidad de los objetos es una característica conveniente: bajo el capó, Pandas toma decisiones sobre el diseño de la memoria y la copia de datos que podrían necesitar realizarse, y el usuario generalmente no necesita preocuparse por estos problemas.

### Serie como matriz unidimensional

Una "Serie" se basa en esta interfaz similar a un diccionario y proporciona selección de elementos de estilo matriz a través de los mismos mecanismos básicos que las matrices NumPy, es decir, cortes, enmascaramiento e indexación elegante.
Ejemplos de estos son los siguientes:

In [7]:
# slicing by explicit index
data['a':'c']

a    0.25
b    0.50
c    0.75
dtype: float64

In [8]:
# slicing by implicit integer index
data[0:2]

a    0.25
b    0.50
dtype: float64

In [9]:
# masking
data[(data > 0.3) & (data < 0.8)]

b    0.50
c    0.75
dtype: float64

In [10]:
# fancy indexing
data[['a', 'e']]

a    0.25
e    1.25
dtype: float64

De estos, el corte puede ser la fuente de mayor confusión.
Tenga en cuenta que cuando se corta con un índice explícito (por ejemplo, `datos['a':'c']`), el índice final se *incluye* en el corte, mientras que cuando se corta con un índice implícito (por ejemplo, `datos[0 :2]`), el índice final está *excluido* del segmento.

### Indexadores: loc e iloc

Si su `Serie` tiene un índice entero explícito, una operación de indexación como `datos[1]` usará los índices explícitos, mientras que una operación de división como `datos[1:3]` usará los índices implícitos estilo Python:

In [11]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

1    a
3    b
5    c
dtype: object

In [12]:
# explicit index when indexing
data[1]

'a'

In [13]:
# implicit index when slicing
data[1:3]

3    b
5    c
dtype: object

Debido a esta posible confusión en el caso de índices enteros, Pandas proporciona algunos atributos *indexador* especiales que exponen explícitamente ciertos esquemas de indexación.
Estos no son métodos funcionales, sino atributos que exponen una interfaz de corte particular a los datos de la "Serie".

Primero, el atributo `loc` permite indexar y dividir que siempre hace referencia al índice explícito:

In [14]:
data.loc[1]

'a'

In [15]:
data.loc[1:3]

1    a
3    b
dtype: object

El atributo `iloc` permite indexar y dividir que siempre hace referencia al índice implícito de estilo Python:

In [16]:
data.iloc[1]

'b'

In [17]:
data.iloc[1:3]

3    b
5    c
dtype: object

Un principio rector del código Python es que "lo explícito es mejor que lo implícito".
La naturaleza explícita de `loc` e `iloc` los hace útiles para mantener un código limpio y legible; especialmente en el caso de índices enteros, usarlos de manera consistente puede evitar errores sutiles debido a la convención mixta de indexación/corte.

## Selección de datos en DataFrames

Recuerde que un `DataFrame` actúa en muchos sentidos como una matriz estructurada o bidimensional, y en otros aspectos como un diccionario de estructuras de `Series` que comparten el mismo índice.
Puede ser útil tener en cuenta estas analogías a medida que exploramos la selección de datos dentro de esta estructura.

### Marco de datos como diccionario

La primera analogía que consideraremos es el `DataFrame` como un diccionario de objetos `Series` relacionados.
Volvamos a nuestro ejemplo de áreas y poblaciones de estados:

In [18]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'Florida': 170312, 'New York': 141297,
                  'Pennsylvania': 119280})
pop = pd.Series({'California': 39538223, 'Texas': 29145505,
                 'Florida': 21538187, 'New York': 20201249,
                 'Pennsylvania': 13002700})
data = pd.DataFrame({'area':area, 'pop':pop})
data

,area,pop
California,423967,39538223
Texas,695662,29145505
Florida,170312,21538187
New York,141297,20201249
Pennsylvania,119280,13002700


Se puede acceder a las "Series" individuales que componen las columnas del "DataFrame" mediante la indexación del nombre de la columna al estilo de un diccionario:

In [19]:
data['area']

California      423967
Texas           695662
Florida         170312
New York        141297
Pennsylvania    119280
Name: area, dtype: int64

De manera equivalente, podemos usar el acceso de estilo de atributo con nombres de columna que sean cadenas:

In [20]:
data.area

California      423967
Texas           695662
Florida         170312
New York        141297
Pennsylvania    119280
Name: area, dtype: int64

Aunque esta es una abreviatura útil, tenga en cuenta que no funciona en todos los casos.
Por ejemplo, si los nombres de las columnas no son cadenas, o si los nombres de las columnas entran en conflicto con los métodos del `DataFrame`, este acceso de estilo de atributo no es posible.
Por ejemplo, `DataFrame` tiene un método `pop`, por lo que `data.pop` apuntará a esto en lugar de a la columna `pop`:

In [21]:
data.pop is data["pop"]

False

En particular, debe evitar la tentación de intentar la asignación de columnas mediante atributos (es decir, usar `data['pop'] = z` en lugar de `data.pop = z`).

Al igual que con los objetos `Series` discutidos anteriormente, esta sintaxis de estilo diccionario también se puede usar para modificar el objeto, en este caso agregando una nueva columna:

In [22]:
data['density'] = data['pop'] / data['area']
data

,area,pop,density
California,423967,39538223,93.257784
Texas,695662,29145505,41.896072
Florida,170312,21538187,126.463121
New York,141297,20201249,142.970120
Pennsylvania,119280,13002700,109.009893


Esto muestra una vista previa de la sintaxis sencilla de la aritmética elemento por elemento entre objetos "Serie"; Profundizaremos en esto en [Operación de datos en Pandas] (03.03-Operaciones-en-Pandas.ipynb).

### Marco de datos como matriz bidimensional

Como se mencionó anteriormente, también podemos ver el `DataFrame` como una matriz bidimensional mejorada.
Podemos examinar la matriz de datos subyacente sin procesar usando el atributo `valores`:

In [23]:
data.values

array([[4.23967000e+05, 3.95382230e+07, 9.32577842e+01],
       [6.95662000e+05, 2.91455050e+07, 4.18960717e+01],
       [1.70312000e+05, 2.15381870e+07, 1.26463121e+02],
       [1.41297000e+05, 2.02012490e+07, 1.42970120e+02],
       [1.19280000e+05, 1.30027000e+07, 1.09009893e+02]])

Con esta imagen en mente, muchas operaciones familiares similares a matrices se pueden realizar en el propio `DataFrame`.
Por ejemplo, podemos transponer el `DataFrame` completo para intercambiar filas y columnas:

In [24]:
data.T

,California,Texas,Florida,New York,Pennsylvania
area,4.239670e+05,6.956620e+05,1.703120e+05,1.412970e+05,1.192800e+05
pop,3.953822e+07,2.914550e+07,2.153819e+07,2.020125e+07,1.300270e+07
density,9.325778e+01,4.189607e+01,1.264631e+02,1.429701e+02,1.090099e+02


Sin embargo, cuando se trata de indexar un objeto `DataFrame`, está claro que la indexación de columnas al estilo de un diccionario impide nuestra capacidad de tratarlo simplemente como una matriz NumPy.
En particular, al pasar un único índice a una matriz se accede a una fila:

In [25]:
data.values[0]

array([4.23967000e+05, 3.95382230e+07, 9.32577842e+01])

y al pasar un único "índice" a un `DataFrame` se accede a una columna:

In [26]:
data['area']

California      423967
Texas           695662
Florida         170312
New York        141297
Pennsylvania    119280
Name: area, dtype: int64

Por tanto, para la indexación estilo matriz, necesitamos otra convención.
Aquí Pandas vuelve a utilizar los indexadores `loc` e `iloc` mencionados anteriormente.
Usando el indexador `iloc`, podemos indexar la matriz subyacente como si fuera una matriz NumPy simple (usando el índice implícito estilo Python), pero el índice `DataFrame` y las etiquetas de las columnas se mantienen en el resultado:

In [27]:
data.iloc[:3, :2]

,area,pop
California,423967,39538223
Texas,695662,29145505
Florida,170312,21538187


De manera similar, usando el indexador `loc` podemos indexar los datos subyacentes en un estilo similar a una matriz pero usando el índice explícito y los nombres de las columnas:

In [28]:
data.loc[:'Florida', :'pop']

,area,pop
California,423967,39538223
Texas,695662,29145505
Florida,170312,21538187


Cualquiera de los patrones familiares de acceso a datos de estilo NumPy se puede utilizar dentro de estos indexadores.
Por ejemplo, en el indexador `loc` podemos combinar enmascaramiento e indexación elegante de la siguiente manera:

In [29]:
data.loc[data.density > 120, ['pop', 'density']]

,pop,density
Florida,21538187,126.463121
New York,20201249,142.970120


Cualquiera de estas convenciones de indexación también se puede utilizar para establecer o modificar valores; Esto se hace de la forma estándar a la que quizás esté acostumbrado al trabajar con NumPy:

In [30]:
data.iloc[0, 2] = 90
data

,area,pop,density
California,423967,39538223,90.000000
Texas,695662,29145505,41.896072
Florida,170312,21538187,126.463121
New York,141297,20201249,142.970120
Pennsylvania,119280,13002700,109.009893


Para desarrollar su fluidez en la manipulación de datos de Pandas, le sugiero dedicar algo de tiempo a un "DataFrame" simple y explorar los tipos de indexación, división, enmascaramiento e indexación sofisticada que permiten estos diversos enfoques de indexación.

### Convenciones de indexación adicionales

Hay un par de convenciones de indexación adicionales que pueden parecer contradictorias con la discusión anterior, pero que, sin embargo, pueden resultar útiles en la práctica.
Primero, mientras que *indexar* se refiere a columnas, *rebanar* se refiere a filas:

In [31]:
data['Florida':'New York']

,area,pop,density
Florida,170312,21538187,126.463121
New York,141297,20201249,142.970120


Estos sectores también pueden referirse a filas por número en lugar de por índice:

In [32]:
data[1:3]

,area,pop,density
Texas,695662,29145505,41.896072
Florida,170312,21538187,126.463121


De manera similar, las operaciones de enmascaramiento directo se interpretan por filas en lugar de por columnas:

In [33]:
data[data.density > 120]

,area,pop,density
Florida,170312,21538187,126.463121
New York,141297,20201249,142.970120


Estas dos convenciones son sintácticamente similares a las de una matriz NumPy y, aunque es posible que no encajen exactamente en el molde de las convenciones de Pandas, se incluyen debido a su utilidad práctica.